## **TITANIC SURVIVOR ANALYSIS**

In [ ]:
import pandas as pd
import numpy as np


# Reading Data Using Pandas



We use pandas *read_csv* to read csv file in python and pandas *Data frame method* to convert file into the data frame

In [ ]:
df=pd.DataFrame(pd.read_csv('/content/train (1).csv'))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.shape

(891, 12)

### **Description of the attributes in dataset**

Pclass : Passenger class(1=1st,2=2nd,3=3rd) 

survival : Survival(0=No,1=Yes)

name : Name


1.   List item
2.   List item


age : Age

sex : Sex

sibsp : Number of siblings/spouses abroad

Parch : Number of parents/children abroad

ticket : Ticket number

fare : passenger Fare(British pound)

cabin : Cabin

embarked :Port of Embarkation(C=Cherbourg ,Q=Qweenstown ,S=Southampton)





# Handling Null Values

The data set may contain rows and columns for which some values are missing.We can't leave them as it is:
We have two options here:
1.we can eliminate entire row or column. 

2.we can fill those missing values by an appropriate value.

In [ ]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

**Now we can eliminate columns which have more than 35% missing  values.**

In [ ]:
x=df.isnull().sum()
drop_col= x[x>(35/100 * df.shape[0])]
drop_col


Cabin    687
dtype: int64

In [ ]:
drop_col.index


Index(['Cabin'], dtype='object')

In [ ]:
df.drop(drop_col.index, axis=1, inplace=True)
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

In [ ]:
df.fillna(df.mean() ,inplace=True)
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

Because *Embarked* has string values in it.We can't get mean of them.So we place more frequently occured string in the missed  place.

In [ ]:
df['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [ ]:
df['Embarked'].fillna('S',inplace=True)

In [ ]:
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [ ]:
df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.033207,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500
Age,0.033207,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566
SibSp,-0.057527,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000


sibsp-Number of siblings and spouses abroad

Parch-Number of parents and children abroad

So we we can combine these two columns to make new column Family_Size

In [ ]:
df['Family_Size']=df['SibSp']+df['Parch']
df.drop(['SibSp','Parch'], axis=1 ,inplace=True)
df.corr()

,PassengerId,Survived,Pclass,Age,Fare,Family_Size
PassengerId,1.000000,-0.005007,-0.035144,0.033207,0.012658,-0.040143
Survived,-0.005007,1.000000,-0.338481,-0.069809,0.257307,0.016639
Pclass,-0.035144,-0.338481,1.000000,-0.331339,-0.549500,0.065997
Age,0.033207,-0.069809,-0.331339,1.000000,0.091566,-0.248512
Fare,0.012658,0.257307,-0.549500,0.091566,1.000000,0.217138
Family_Size,-0.040143,0.016639,0.065997,-0.248512,0.217138,1.000000


Family_Size  in the ship does not have much correlance with the survival rate.

Let's check whether the person was alone or not can effect the survival rate.

In [ ]:
df['Alone']=[0 if df['Family_Size'][i]>0 else 1 for i in df.index]
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,Family_Size,Alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,S,0,1


In [ ]:
df.groupby(['Alone'])['Survived'].mean()

Alone
0    0.505650
1    0.303538
Name: Survived, dtype: float64

If person is alone,there is less chance for surviving.

Reason might be person with family might be rich and have high prioritized.

In [ ]:
df[['Alone','Fare']].corr()

,Alone,Fare
Alone,1.000000,-0.271832
Fare,-0.271832,1.000000


So if person is not alone,the chance the ticket price is higher is high.

In [ ]:
df['Sex']= [0 if df['Sex'][i]=="male" else 1 for i in df.index] # 0 for male,1 for female
df.groupby(['Sex'])['Survived'].mean()

Sex
0    0.188908
1    0.742038
Name: Survived, dtype: float64

It shows,Females Survival rate is more than males

In [ ]:
df.groupby(['Embarked'])['Survived'].mean()

Embarked
C    0.553571
Q    0.389610
S    0.339009
Name: Survived, dtype: float64

Embarkation of Cherbourg passengers survived more.

# **Conclusion**

*   Female passengers were priortized over men.
*   People with rich class or higher class have high survival rate than others.The hierarchy might be followed in saving passengers.
*   Passengers with family members have high survival rate.
*   Passengers who boarded in Cherbourg ,survived more in proportion than others.



